In [6]:
import csv
import json
import random
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

In [42]:
# model = SentenceTransformer('llmrails/ember-v1')
model = SentenceTransformer('WhereIsAI/UAE-Large-V1')

No sentence-transformers model found with name WhereIsAI/UAE-Large-V1. Creating a new one with MEAN pooling.


config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [43]:
sentences = [
    "This is an example sentence",
    "Each sentence is converted"
]
embeddings = model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

tensor([[0.7438]])


In [44]:
# load the meddra terms
fh = open('data/meddra_llt_pt_map.txt')
reader = csv.reader(fh, delimiter='|')
header = next(reader)

meddra_llt_terms = set()
meddra_pt_terms = set()

for row in reader:
    meddra_llt_terms.add(row[1].lower())
    meddra_pt_terms.add(row[4].lower())

fh.close()

meddra_terms = sorted(meddra_llt_terms | meddra_pt_terms)
len(meddra_terms)

73302

In [45]:
sampled_meddra_terms = random.sample(meddra_terms, 100) + ["Hyperglycaemia"]
sampled_embeddings = model.encode(sampled_meddra_terms)

In [46]:
query = "high blood glucose"
query_embedding = model.encode(query)
query_embedding

array([-0.21122518, -0.05385017, -0.02172772, ...,  0.0987644 ,
        0.18727395,  0.04142275], dtype=float32)

In [47]:
cos_sims = list()
for i in range(sampled_embeddings.shape[0]):
    cos_sims.append(float(cos_sim(query_embedding, sampled_embeddings[i])))

In [48]:
sampled_meddra_terms[np.argmax(cos_sims)], np.max(cos_sims)

('Hyperglycaemia', 0.8582732081413269)

In [11]:
from Llamasgard import CodeLlama
from openai import OpenAI, AzureOpenAI
import json

In [15]:
section_text = """6 ADVERSE REACTIONS

  Because clinical studies are conducted under widely varying conditions, adverse reaction rates observed in the clinical studies of a drug cannot be directly compared to rates in the clinical studies of another drug and may not predict the rates observed in a broader patient population in clinical practice.



   EXCERPT:   Most common adverse reactions (incidence of at least 5%): upper respiratory tract infections, nasopharyngitis, headache, hypertension, increased ALT, injection site reactions. (  6.1  )  To report SUSPECTED ADVERSE REACTIONS, contact Genentech at 1-888-835-2555 or FDA at 1-800-FDA-1088 or www.fda.gov/medwatch  



 

  6.1 Clinical Trials Experience in Rheumatoid Arthritis Patients Treated with Intravenous ACTEMRA (ACTEMRA-IV)

  The ACTEMRA-IV data in rheumatoid arthritis (RA) includes 5 double-blind, controlled, multicenter studies. In these studies, patients received doses of ACTEMRA-IV 8 mg per kg monotherapy (288 patients), ACTEMRA-IV 8 mg per kg in combination with DMARDs (including methotrexate) (1582 patients), or ACTEMRA-IV 4 mg per kg in combination with methotrexate (774 patients).



 The all exposure population includes all patients in registration studies who received at least one dose of ACTEMRA-IV. Of the 4009 patients in this population, 3577 received treatment for at least 6 months, 3309 for at least one year; 2954 received treatment for at least 2 years and 2189 for 3 years.



 All patients in these studies had moderately to severely active rheumatoid arthritis. The study population had a mean age of 52 years, 82% were female and 74% were Caucasian.



 The most common serious adverse reactions were serious infections  [see  Warnings and Precautions (5.1)  ]  . The most commonly reported adverse reactions in controlled studies up to 24 weeks (occurring in at least 5% of patients treated with ACTEMRA-IV monotherapy or in combination with DMARDs) were upper respiratory tract infections, nasopharyngitis, headache, hypertension and increased ALT.



 The proportion of patients who discontinued treatment due to any adverse reactions during the double-blind, placebo-controlled studies was 5% for patients taking ACTEMRA-IV and 3% for placebo-treated patients. The most common adverse reactions that required discontinuation of ACTEMRA-IV were increased hepatic transaminase values (per protocol requirement) and serious infections.



     Overall Infections  



 In the 24 week, controlled clinical studies, the rate of infections in the ACTEMRA-IV monotherapy group was 119 events per 100 patient-years and was similar in the methotrexate monotherapy group. The rate of infections in the 4 mg per kg and 8 mg per kg ACTEMRA-IV plus DMARD group was 133 and 127 events per 100 patient-years, respectively, compared to 112 events per 100 patient-years in the placebo plus DMARD group. The most commonly reported infections (5% to 8% of patients) were upper respiratory tract infections and nasopharyngitis.



 The overall rate of infections with ACTEMRA-IV in the all exposure population remained consistent with rates in the controlled periods of the studies.



     Serious Infections  



 In the 24 week, controlled clinical studies, the rate of serious infections in the ACTEMRA-IV monotherapy group was 3.6 per 100 patient-years compared to 1.5 per 100 patient-years in the methotrexate group. The rate of serious infections in the 4 mg per kg and 8 mg per kg ACTEMRA-IV plus DMARD group was 4.4 and 5.3 events per 100 patient-years, respectively, compared to 3.9 events per 100 patient-years in the placebo plus DMARD group.



 In the all-exposure population, the overall rate of serious infections remained consistent with rates in the controlled periods of the studies. The most common serious infections included pneumonia, urinary tract infection, cellulitis, herpes zoster, gastroenteritis, diverticulitis, sepsis and bacterial arthritis. Cases of opportunistic infections have been reported  [see  Warnings and Precautions (5.1)  ]  .



     Gastrointestinal Perforations  



 During the 24 week, controlled clinical trials, the overall rate of gastrointestinal perforation was 0.26 events per 100 patient-years with ACTEMRA-IV therapy.



 In the all-exposure population, the overall rate of gastrointestinal perforation remained consistent with rates in the controlled periods of the studies. Reports of gastrointestinal perforation were primarily reported as complications of diverticulitis including generalized purulent peritonitis, lower GI perforation, fistula and abscess. Most patients who developed gastrointestinal perforations were taking concomitant nonsteroidal anti-inflammatory medications (NSAIDs), corticosteroids, or methotrexate  [see  Warnings and Precautions (5.2)  ]  . The relative contribution of these concomitant medications versus ACTEMRA-IV to the development of GI perforations is not known.



       Infusion Reactions    



 In the 24 week, controlled clinical studies, adverse events associated with the infusion (occurring during or within 24 hours of the start of infusion) were reported in 8% and 7% of patients in the 4 mg per kg and 8 mg per kg ACTEMRA-IV plus DMARD group, respectively, compared to 5% of patients in the placebo plus DMARD group. The most frequently reported event on the 4 mg per kg and 8 mg per kg dose during the infusion was hypertension (1% for both doses), while the most frequently reported event occurring within 24 hours of finishing an infusion were headache (1% for both doses) and skin reactions (1% for both doses), including rash, pruritus and urticaria. These events were not treatment limiting.



       Anaphylaxis    



 Hypersensitivity reactions requiring treatment discontinuation, including anaphylaxis, associated with ACTEMRA-IV were reported in 0.1% (3 out of 2644) in the 24 week, controlled trials and in 0.2% (8 out of 4009) in the all-exposure population. These reactions were generally observed during the second to fourth infusion of ACTEMRA-IV. Appropriate medical treatment should be available for immediate use in the event of a serious hypersensitivity reaction  [see  Warnings and Precautions (5.5)  ]  .



     Laboratory Abnormalities  



     Neutropenia  



 In the 24 week, controlled clinical studies, decreases in neutrophil counts below 1000 per mm  3  occurred in 1.8% and 3.4% of patients in the 4 mg per kg and 8 mg per kg ACTEMRA-IV plus DMARD group, respectively, compared to 0.1% of patients in the placebo plus DMARD group. Approximately half of the instances of ANC below 1000 per mm  3  occurred within 8 weeks of starting therapy. Decreases in neutrophil counts below 500 per mm  3  occurred in 0.4% and 0.3% of patients in the 4 mg per kg and 8 mg per kg ACTEMRA-IV plus DMARD, respectively, compared to 0.1% of patients in the placebo plus DMARD group. There was no clear relationship between decreases in neutrophils below 1000 per mm  3  and the occurrence of serious infections.



 In the all-exposure population, the pattern and incidence of decreases in neutrophil counts remained consistent with what was seen in the 24 week controlled clinical studies  [see  Warnings and Precautions (5.3)  ]  .



     Thrombocytopenia  



 In the 24 week, controlled clinical studies, decreases in platelet counts below 100,000 per mm  3  occurred in 1.3% and 1.7% of patients on 4 mg per kg and 8 mg per kg ACTEMRA-IV plus DMARD, respectively, compared to 0.5% of patients on placebo plus DMARD, without associated bleeding events.



 In the all-exposure population, the pattern and incidence of decreases in platelet counts remained consistent with what was seen in the 24 week controlled clinical studies  [see  Warnings and Precautions (5.3)  ]  .



     Elevated Liver Enzymes  



 Liver enzyme abnormalities are summarized in    Table 1    . In patients experiencing liver enzyme elevation, modification of treatment regimen, such as reduction in the dose of concomitant DMARD, interruption of ACTEMRA-IV, or reduction in ACTEMRA-IV dose, resulted in decrease or normalization of liver enzymes  [see  Dosage and Administration (2.5)  ]  . These elevations were not associated with clinically relevant increases in direct bilirubin, nor were they associated with clinical evidence of hepatitis or hepatic insufficiency  [see  Warnings and Precautions (5.3)  ]  .



 Table 1 Incidence of Liver Enzyme Abnormalities in the 24 Week Controlled Period of Studies I to VFor a description of these studies, see Section 14, Clinical Studies. 
                       ACTEMRA8 mg per kg MONOTHERAPY     Methotrexate    ACTEMRA4 mg per kg + DMARDs  ACTEMRA8 mg per kg + DMARDs  Placebo + DMARDs   
                          N = 288(%)        N = 284(%)        N = 774(%)       N = 1582(%)       N = 1170(%)      
  
 ULN = Upper Limit of Normal   
  
 AST (U/L)                                                                                                        
 &gt; ULN to 3* ULN              22                26                34                41                17          
 &gt; 3* ULN to 5* ULN          0.3                2                 1                 2                0.3          
 &gt; 5* ULN                    0.7               0.4               0.1               0.2              &lt; 0.1         
 ALT (U/L)                                                                                                        
 &gt; ULN to 3* ULN              36                33                45                48                23          
 &gt; 3* ULN to 5* ULN           1                 4                 5                 5                 1           
 &gt; 5* ULN                    0.7                1                1.3               1.5               0.3          
            In the all-exposure population, the elevations in ALT and AST remained consistent with what was seen in the 24 week, controlled clinical trials
 

     Lipids  



 Elevations in lipid parameters (total cholesterol, LDL, HDL, triglycerides) were first assessed at 6 weeks following initiation of ACTEMRA-IV in the controlled 24 week clinical trials. Increases were observed at this time point and remained stable thereafter. Increases in triglycerides to levels above 500 mg per dL were rarely observed. Changes in other lipid parameters from baseline to week 24 were evaluated and are summarized below:



 *  -  Mean LDL increased by 13 mg per dL in the ACTEMRA 4 mg per kg+DMARD arm, 20 mg per dL in the ACTEMRA 8 mg per kg+DMARD, and 25 mg per dL in ACTEMRA 8 mg per kg monotherapy. 
 *  -  Mean HDL increased by 3 mg per dL in the ACTEMRA 4 mg per kg+DMARD arm, 5 mg per dL in the ACTEMRA 8 mg per kg+DMARD, and 4 mg per dL in ACTEMRA 8 mg per kg monotherapy. 
 *  -  Mean LDL/HDL ratio increased by an average of 0.14 in the ACTEMRA 4 mg per kg+DMARD arm, 0.15 in the ACTEMRA 8 mg per kg+DMARD, and 0.26 in ACTEMRA 8 mg per kg monotherapy. 
 *  -  ApoB/ApoA1 ratios were essentially unchanged in ACTEMRA-treated patients. 
    Elevated lipids responded to lipid lowering agents.
 

 In the all-exposure population, the elevations in lipid parameters remained consistent with what was seen in the 24 week, controlled clinical trials.



       Immunogenicity    



 In the 24 week, controlled clinical studies, a total of 2876 patients have been tested for anti-tocilizumab antibodies. Forty-six patients (2%) developed positive anti-tocilizumab antibodies, of whom 5 had an associated, medically significant, hypersensitivity reaction leading to withdrawal. Thirty patients (1%) developed neutralizing antibodies.



 The data reflect the percentage of patients whose test results were positive for antibodies to tocilizumab in specific assays. The observed incidence of antibody positivity in an assay is highly dependent on several factors, including assay sensitivity and specificity, assay methodology, sample handling, timing of sample collection, concomitant medication, and underlying disease. For these reasons, comparison of the incidence of antibodies to tocilizumab with the incidence of antibodies to other products may be misleading.



       Malignancies    



 During the 24 week, controlled period of the studies, 15 malignancies were diagnosed in patients receiving ACTEMRA-IV, compared to 8 malignancies in patients in the control groups. Exposure-adjusted incidence was similar in the ACTEMRA-IV groups (1.32 events per 100 patient-years) and in the placebo plus DMARD group (1.37 events per 100 patient-years).



 In the all-exposure population, the rate of malignancies remained consistent with the rate observed in the 24 week, controlled period  [see  Warnings and Precautions (5.4)  ]  .



       Other Adverse Reactions    



 Adverse reactions occurring in 2% or more of patients on 4 or 8 mg per kg ACTEMRA-IV plus DMARD and at least 1% greater than that observed in patients on placebo plus DMARD are summarized in    Table 2  .  



 Table 2 Adverse Reactions Occurring in at Least 2% or More of Patients on 4 or 8 mg per kg ACTEMRA plus DMARD and at Least 1% Greater Than That Observed in Patients on Placebo plus DMARD 
 24 Week Phase 3 Controlled Study Population   
                                 ACTEMRA8 mg per kg MONOTHERAPY    Methotrexate   ACTEMRA4 mg per kg + DMARDs  ACTEMRA8 mg per kg + DMARDs  Placebo + DMARDs   
 Preferred Term                     N = 288(%)       N = 284(%)      N = 774(%)     N = 1582(%)    N = 1170(%)    
  
 Upper Respiratory Tract Infection         7                5               6               8              6         
 Nasopharyngitis                        7                6               4               6              4         
 Headache                               7                2               6               5              3         
 Hypertension                           6                2               4               4              3         
 ALT increased                          6                4               3               3              1         
 Dizziness                              3                1               2               3              2         
 Bronchitis                             3                2               4               3              3         
 Rash                                   2                1               4               3              1         
 Mouth Ulceration                       2                2               1               2              1         
 Abdominal Pain Upper                   2                2               3               3              2         
 Gastritis                              1                2               1               2              1         
 Transaminase increased                 1                5               2               2              1         
            Other infrequent and medically relevant adverse reactions occurring at an incidence less than 2% in rheumatoid arthritis patients treated with ACTEMRA-IV in controlled trials were:
 

   Infections and Infestations:  oral herpes simplex



   Gastrointestinal disorders:  stomatitis, gastric ulcer



   Investigations:  weight increased, total bilirubin increased



   Blood and lymphatic system disorders:  leukopenia



   General disorders and administration site conditions:  edema peripheral



   Respiratory, thoracic, and mediastinal disorders  : dyspnea, cough



   Eye disorders:  conjunctivitis



   Renal disorders:  nephrolithiasis



   Endocrine disorders:  hypothyroidism



   6.2 Clinical Trials Experience in Rheumatoid Arthritis Patients Treated with Subcutaneous ACTEMRA (ACTEMRA-SC)

  The ACTEMRA-SC data in rheumatoid arthritis (RA) includes 2 double-blind, controlled, multicenter studies. Study SC-I was a non-inferiority study that compared the efficacy and safety of tocilizumab 162 mg administered every week subcutaneously (SC) and 8 mg/kg intravenously (IV) every four weeks in 1262 adult subjects with rheumatoid arthritis. Study SC-II was a placebo controlled superiority study that evaluated the safety and efficacy of tocilizumab 162 mg administered every other week SC or placebo in 656 patients. All patients in both studies received background non-biologic DMARDs.



 The safety observed for ACTEMRA administered subcutaneously was consistent with the known safety profile of intravenous ACTEMRA, with the exception of injection site reactions, which were more common with ACTEMRA-SC compared with placebo SC injections (IV arm).



       Injection Site Reactions    



 In the 6-month control period, in SC-I, the frequency of injection site reactions was 10.1% (64/631) and 2.4% (15/631) for the weekly ACTEMRA-SC and placebo SC (IV-arm) groups, respectively. In SC-II, the frequency of injection site reactions was 7.1% (31/437) and 4.1% (9/218) for the every other week SC ACTEMRA and placebo groups, respectively. These injection site reactions (including erythema, pruritus, pain and hematoma) were mild to moderate in severity. The majority resolved without any treatment and none necessitated drug discontinuation.



       Immunogenicity    



 In the 6-month control period in SC-I, 0.8% (5/625) in the ACTEMRA-SC arm and 0.8% (5/627) in the IV arm developed anti-tocilizumab antibodies; of these, all developed neutralizing antibodies. In SC-II, 1.6% (7/434) in the ACTEMRA-SC arm compared with 1.4 % (3/217) in the placebo arm developed anti- tocilizumab antibodies; of these, 1.4% (6/434) in the ACTEMRA-SC arm and 0.5% (1/217) in the placebo arm also developed neutralizing antibodies.



 A total of 1454 (&gt;99%) patients who received ACTEMRA-SC in the all exposure group have been tested for anti-tocilizumab antibodies. Thirteen patients (0.9%) developed anti-tocilizumab antibodies, and, of these, 12 patients (0.8%) developed neutralizing antibodies.



 The rate is consistent with previous intravenous experience. No correlation of antibody development to adverse events or loss of clinical response was observed.



       Laboratory Abnormalities    



     Neutropenia  



 During routine laboratory monitoring in the 6-month controlled clinical trials, a decrease in neutrophil count below 1 * 10  9  /L occurred in 2.9% and 3.7% of patients receiving ACTEMRA-SC weekly and every other week, respectively.



 There was no clear relationship between decreases in neutrophils below 1 * 10  9  /L and the occurrence of serious infections.



     Thrombocytopenia  



 During routine laboratory monitoring in the ACTEMRA-SC 6-month controlled clinical trials, none of the patients had a decrease in platelet count to &lt;=50 * 10  3  /mcL.



     Elevated Liver Enzymes  



 During routine laboratory monitoring in the 6-month controlled clinical trials, elevation in ALT or AST &gt;=3 * ULN occurred in 6.5% and 1.4% of patients, respectively, receiving ACTEMRA-SC weekly and 3.4% and 0.7% receiving ACTEMRA SC every other week.



     Lipids  



 During routine laboratory monitoring in the ACTEMRA-SC 6-month clinical trials, 19% of patients dosed weekly and 19.6% of patients dosed every other week and 10.2% of patients on placebo experienced sustained elevations in total cholesterol &gt; 6.2 mmol/l (240 mg/dL), with 9%, 10.4% and 5.1% experiencing a sustained increase in LDL to 4.1 mmol/l (160 mg/dL) receiving ACTEMRA-SC weekly, every other week and placebo, respectively.



   6.3 Clinical Trials Experience in Polyarticular Juvenile Idiopathic Arthritis Patients Treated With Intravenous ACTEMRA (ACTEMRA-IV)

  The safety of ACTEMRA-IV was studied in 188 pediatric patients 2 to 17 years of age with PJIA who had an inadequate clinical response or were intolerant to methotrexate. The total patient exposure in the ACTEMRA-IV all exposure population (defined as patients who received at least one dose of ACTEMRA-IV) was 184.4 patient years. At baseline, approximately half of the patients were taking oral corticosteroids and almost 80% were taking methotrexate. In general, the types of adverse drug reactions in patients with PJIA were consistent with those seen in RA and SJIA patients [  see  Adverse Reactions (6.1  and  6.4)    ].



     Infections  



 The rate of infections in the ACTEMRA-IV all exposure population was 163.7 per 100 patient years. The most common events observed were nasopharyngitis and upper respiratory tract infections. The rate of serious infections was numerically higher in patients weighing less than 30 kg treated with 10 mg/kg tocilizumab (12.2 per 100 patient years) compared to patients weighing at or above 30 kg, treated with 8 mg/kg tocilizumab (4.0 per 100 patient years). The incidence of infections leading to dose interruptions was also numerically higher in patients weighing less than 30 kg treated with 10 mg/kg tocilizumab (21%) compared to patients weighing at or above 30 kg, treated with 8 mg/kg tocilizumab (8%).



     Infusion Reactions  



 In PJIA patients, infusion-related reactions are defined as all events occurring during or within 24 hours of an infusion. In the ACTEMRA-IV all exposure population, 11 patients (6%) experienced an event during the infusion, and 38 patients (20.2%) experienced an event within 24 hours of an infusion. The most common events occurring during infusion were headache, nausea and hypotension, and occurring within 24 hours of infusion were dizziness and hypotension. In general, the adverse drug reactions observed during or within 24 hours of an infusion were similar in nature to those seen in RA and SJIA patients  [see  Adverse Reactions (6.1  and  6.4)  ]  .



 No clinically significant hypersensitivity reactions associated with tocilizumab and requiring treatment discontinuation were reported.



     Immunogenicity  



 One patient, in the 10 mg/kg less than 30 kg group, developed positive anti-tocilizumab antibodies without developing a hypersensitivity reaction and subsequently withdrew from the study.



     Laboratory Abnormalities  



     Neutropenia  



 During routine laboratory monitoring in the ACTEMRA-IV all exposure population, a decrease in neutrophil counts below 1 * 10  9  per L occurred in 3.7% of patients.



 There was no clear relationship between decreases in neutrophils below 1 * 10  9  per L and the occurrence of serious infections.



     Thrombocytopenia  



 During routine laboratory monitoring in the ACTEMRA-IV all exposure population, 1% of patients had a decrease in platelet count at or less than 50 * 10  3  per mcL without associated bleeding events.



     Elevated Liver Enzymes  



 During routine laboratory monitoring in the ACTEMRA-IV all exposure population, elevation in ALT or AST at or greater than 3 * ULN occurred in 4% and less than 1% of patients, respectively.



     Lipids  



 During routine laboratory monitoring in the tocilizumab all exposure population, elevation in total cholesterol greater than 1.5-2 * ULN occurred in one patient (0.5%) and elevation in LDL greater than 1.5-2 * ULN occurred in one patient (0.5%).



   6.4 Clinical Trials Experience in Systemic Juvenile Idiopathic Arthritis Patients Treated with Intravenous ACTEMRA (ACTEMRA-IV)

  The data described below reflect exposure to ACTEMRA-IV in one randomized, double-blind, placebo-controlled trial of 112 pediatric patients with SJIA 2 to 17 years of age who had an inadequate clinical response to nonsteroidal anti-inflammatory drugs (NSAIDs) or corticosteroids due to toxicity or lack of efficacy. At baseline, approximately half of the patients were taking 0.3 mg/kg/day corticosteroids or more, and almost 70% were taking methotrexate. The trial included a 12 week controlled phase followed by an open-label extension. In the 12 week double-blind, controlled portion of the clinical study 75 patients received treatment with ACTEMRA-IV (8 or 12 mg per kg based upon body weight). After 12 weeks or at the time of escape, due to disease worsening, patients were treated with ACTEMRA-IV in the open-label extension phase.



 The most common adverse events (at least 5%) seen in ACTEMRA-IV treated patients in the 12 week controlled portion of the study were: upper respiratory tract infection, headache, nasopharyngitis and diarrhea.



       Infections    



 In the 12 week controlled phase, the rate of all infections in the ACTEMRA-IV group was 345 per 100 patient-years and 287 per 100 patient-years in the placebo group. In the open label extension over an average duration of 73 weeks of treatment, the overall rate of infections was 304 per 100 patient-years.



 In the 12 week controlled phase, the rate of serious infections in the ACTEMRA-IV group was 11.5 per 100 patient years. In the open label extension over an average duration of 73 weeks of treatment, the overall rate of serious infections was 11.4 per 100 patient years. The most commonly reported serious infections included pneumonia, gastroenteritis, varicella, and otitis media.



       Macrophage Activation Syndrome    



 In the 12 week controlled study, no patient in any treatment group experienced macrophage activation syndrome (MAS) while on assigned treatment; 3 per 112 (3%) developed MAS during open-label treatment with ACTEMRA-IV. One patient in the placebo group escaped to ACTEMRA-IV 12 mg per kg at Week 2 due to severe disease activity, and ultimately developed MAS at Day 70. Two additional patients developed MAS during the long-term extension. All 3 patients had ACTEMRA-IV dose interrupted (2 patients) or discontinued (1 patient) for the MAS event, received treatment, and the MAS resolved without sequelae. Based on a limited number of cases, the incidence of MAS does not appear to be elevated in the ACTEMRA-IV SJIA clinical development experience; however no definitive conclusions can be made.



       Infusion Reactions    



 Patients were not premedicated, however most patients were on concomitant corticosteroids as part of their background treatment for SJIA. Infusion related reactions were defined as all events occurring during or within 24 hours after an infusion. In the 12 week controlled phase, 4% of ACTEMRA-IV and 0% of placebo treated patients experienced events occurring during infusion. One event (angioedema) was considered serious and life-threatening, and the patient was discontinued from study treatment.



 Within 24 hours after infusion, 16% of patients in the ACTEMRA-IV treatment group and 5% of patients in the placebo group experienced an event. In the ACTEMRA-IV group the events included rash, urticaria, diarrhea, epigastric discomfort, arthralgia and headache. One of these events, urticaria, was considered serious.



       Anaphylaxis    



 Anaphylaxis was reported in 1 out of 112 patients (less than 1%) treated with ACTEMRA-IV during the controlled and open label extension study  [see  Warnings and Precautions (5.5)  ]  .



       Immunogenicity    



 All 112 patients were tested for anti-tocilizumab antibodies at baseline. Two patients developed positive anti-tocilizumab antibodies: one of these patients experienced serious adverse events of urticaria and angioedema consistent with an anaphylactic reaction which led to withdrawal; the other patient developed macrophage activation syndrome while on escape therapy and was discontinued from the study.



     Laboratory Abnormalities  



     Neutropenia  



 During routine monitoring in the 12 week controlled phase, a decrease in neutrophil below 1 * 10  9  per L occurred in 7% of patients in the ACTEMRA-IV group, and in no patients in the placebo group. In the open label extension over an average duration of 73 weeks of treatment, a decreased neutrophil count occurred in 17% of the ACTEMRA-IV group. There was no clear relationship between decrease in neutrophils below 1 * 10  9  per L and the occurrence of serious infections.



     Thrombocytopenia  



 During routine monitoring in the 12 week controlled phase, 1% of patients in the ACTEMRA-IV group and 3% in the placebo group had a decrease in platelet count to no more than 100 * 10  3  per mcL.



 In the open label extension over an average duration of 73 weeks of treatment, decreased platelet count occurred in 4% of patients in the ACTEMRA-IV group, with no associated bleeding.



     Elevated Liver Enzymes  



 During routine laboratory monitoring in the 12 week controlled phase, elevation in ALT or AST at or above 3* ULN occurred in 5% and 3% of patients, respectively in the ACTEMRA-IV group and in 0% of placebo patients.



 In the open label extension over an average duration of 73 weeks of treatment, the elevation in ALT or AST at or above 3* ULN occurred in 13% and 5% of ACTEMRA-IV treated patients, respectively.



     Lipids  



 During routine laboratory monitoring in the 12 week controlled phase, elevation in total cholesterol greater than 1.5* ULN - 2x ULN occurred in 1.5% of the ACTEMRA-IV group and in 0% of placebo patients. Elevation in LDL greater than 1.5* ULN - 2x ULN occurred in 1.9% of patients in the ACTEMRA-IV group and 0% of the placebo group.



 In the open label extension study over an average duration of 73 weeks of treatment, the pattern and incidence of elevations in lipid parameters remained consistent with the 12 week controlled study data.



   6.5 Postmarketing Experience

  The following adverse reactions have been identified during postapproval use of intravenous ACTEMRA. Because these reactions are reported voluntarily from a population of uncertain size, it is not always possible to reliably estimate their frequency or establish a causal relationship to drug exposure.



 *  Fatal anaphylaxis [see  Warnings and Precautions (5.5)  ]  
 *  Stevens-Johnson Syndrome"""

In [21]:
len(section_text)

29791

In [24]:
llm=CodeLlama(system="You are a pharmacology expert.", temperature=0, max_new_tokens=2048)

prompt = """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
The text is : {}
 """.format(section_text[:15000])
response = llm(prompt=prompt)
print(response)

 Here is a list of adverse reactions mentioned in the text, along with their corresponding preferred terms:

* Upper respiratory tract infections (URIs)
* Nasopharyngitis
* Headache
* Hypertension
* Increased ALT
* Death (as a result of severe infections)
* Injection site reactions
* Gastrointestinal perforation
* Opportunistic infections
* Severe infections (such as pneumonia, urinary tract infection, cellulitis, herpes zoster, gastroenteritis, diverticulitis, sepsis, and bacterial arthritis)
* Gastrointestral perforation
* Skin reactions (rash, pruritus, urticaria)
* Anaphylaxis
* Neutropenia
* Thrombocytopenia
* Elevated liver enzymes (AST and ALT)
* Lipid changes (triglyceride increase, LDL increase, HDL increase, LDL/HDL ratio increase)
* Anti-tocilizumab antibodies
* Neutralizing antibodies
* Malignancies
* Other adverse reactions (dizziness, bronchitis, mouth ulceration, abdominal pain upper, gastritis)


In [4]:
def perform_cleanup(extraction, openai_api):
    client = OpenAI(api_key=openai_api)
    
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": ""},
            {
                "role": "user",
                "content": """The following text is an extraction of adverse event terms from a drug label. Please remove any preamble or postamble from the list and turn the list of ADEs into a comma separated list. 
The text: {}""".format(extraction)
            }
        ],
        model="gpt-3.5-turbo-16k",
        temperature=0,
    )
    term = chat_completion.choices[0].message.content
    return term

In [7]:
config = json.load(open('./config.json'))

In [25]:
perform_cleanup(response, config['OpenAI']['openai_api_key'])

'Upper respiratory tract infections (URIs), Nasopharyngitis, Headache, Hypertension, Increased ALT, Death (as a result of severe infections), Injection site reactions, Gastrointestinal perforation, Opportunistic infections, Severe infections (such as pneumonia, urinary tract infection, cellulitis, herpes zoster, gastroenteritis, diverticulitis, sepsis, and bacterial arthritis), Gastrointestral perforation, Skin reactions (rash, pruritus, urticaria), Anaphylaxis, Neutropenia, Thrombocytopenia, Elevated liver enzymes (AST and ALT), Lipid changes (triglyceride increase, LDL increase, HDL increase, LDL/HDL ratio increase), Anti-tocilizumab antibodies, Neutralizing antibodies, Malignancies, Other adverse reactions (dizziness, bronchitis, mouth ulceration, abdominal pain upper, gastritis)'